In [1]:
#import libraries
import json
import numpy as np
from sklearn.externals import joblib

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from collections import defaultdict

In [2]:
# load predict topic model
import predict_topic
predict_topic.init('nmf_model.joblib', 'vectorizer.joblib')

# load predict sentiment model
import predict_sentiment


In [6]:
# load data and set #data
total_data= 2000
data_filename= 'restaurant_review.json'

In [7]:
# returns topic and sentiment dictionary {D[topic#]['SA'] = #} per review
def rating_per_review(review):
    #print review
    d = {}
    i = 0
    for sentence in review.split("."):
        SA = predict_sentiment.get_sentiment(sentence)[0]
        i += 1
        topic = predict_topic.predict_topic(sentence)
        d[topic] = d.get(topic,{})
        d[topic][SA] = d[topic].get(SA,0)
        d[topic][SA] += 1   
    return d    

In [8]:
# for given data, returns average rating of a business considering both topic and sentiment analysis
def rating_per_business(documents):
    #{ i : 0 for i in range(4) }
    final_d = {}
    aggregate_rating = { i : 0.0 for i in range(5) }
    for review in documents:
        r = rating_per_review(review["text"])
        advanced_r = r
        
        # combine ratings for all reviews of a business
        for topic in r:
            final_d[topic] = final_d.get(topic,{})           
            for SA in r[topic]:  
                final_d[topic][SA] = final_d[topic].get(SA,0)
                final_d[topic][SA] += r[topic][SA]
    
    # find the average SA per topic (P / P + N)
    for topic in final_d:
               # print d[topic]
            positive = final_d[topic]['P'] if 'P' in final_d[topic] else 0
            negative = final_d[topic]['N'] if 'N' in final_d[topic] else 0

            aggregate_rating[topic] = (positive*1.0)/(positive+ negative)*1.0
    # return aggregate rating for entire business
    # {0: 1.0, 1: 0.8571428571428571, 2: 0.0, 3: 1.0, 4: 0.0}
    return aggregate_rating  


In [9]:
# load data
with open(data_filename) as f:
        head = [json.loads(next(f)) for x in range(total_data)]
        
businesses = {}
users = {}
users_business_topic_rating =defaultdict(list)
business_id_map ={}
id_business_map ={}
count_business = 0
count_user = 0
user_id_map = {}
id_user_map = {}


# loop through each review in the data
for i in head:
    b_id = i["business_id"]
    # create business->id and id->business map
    if b_id not in business_id_map:
        business_id_map[b_id] = count_business 
        id_business_map[count_business] = b_id
        count_business += 1
    # businesses contains all the reviews for a given business
    # each businesses[b_id] will have an array of reviews for that particular b_id
    businesses[b_id] = businesses.get(b_id, [])
    businesses[b_id].append(i)
    
    # create user->id and id->user map
    u_id = i["user_id"]
    if u_id not in user_id_map:
        user_id_map[u_id] = count_user 
        id_user_map[count_user] = u_id
        count_user += 1
    # users contains all the reviews given by a user
    # each users[u_id] will have an array of reviews given by that particular u_id
    users[u_id] = users.get(u_id, [])
    users[u_id].append(i)
    text = i["text"]
    
    # for each review, get the average rating per topic (based on sentiment analysis)
    rating_for_topic = rating_per_business([i])
    # loop through topics of the particular review
    for topic in rating_for_topic:
        # store per topic rating separately, topic: [[user, business, rating]]
        # {0: [[0, 0, 0.5],
        #      [0, 1, 1.0]...
        users_business_topic_rating[topic].append([user_id_map[u_id],business_id_map[b_id],rating_for_topic[topic]])


In [10]:
users_business_topic_rating

defaultdict(list,
            {0: [[0, 0, 0.5],
              [0, 1, 1.0],
              [0, 2, 0.5],
              [0, 3, 0.0],
              [0, 4, 1.0],
              [0, 5, 0.25],
              [0, 6, 0.8],
              [0, 7, 1.0],
              [0, 8, 1.0],
              [0, 9, 0.5],
              [1, 10, 0.0],
              [2, 11, 0.0],
              [3, 12, 0.0],
              [3, 13, 1.0],
              [3, 14, 1.0],
              [4, 15, 1.0],
              [4, 16, 0.0],
              [4, 17, 0.0],
              [4, 18, 1.0],
              [5, 19, 1.0],
              [6, 20, 0.0],
              [6, 21, 0.0],
              [7, 22, 1.0],
              [8, 23, 0.5],
              [8, 24, 0.0],
              [8, 25, 1.0],
              [9, 26, 0.0],
              [10, 27, 0.0],
              [11, 28, 0.25],
              [12, 29, 1.0],
              [13, 30, 1.0],
              [14, 31, 1.0],
              [14, 32, 0.0],
              [14, 33, 0.0],
              [14, 34, 1.0],

In [11]:
business_rating_per_topic = defaultdict(dict)
# get average rating per business
for x in businesses:
    business_rating_per_topic[x] = rating_per_business(businesses[x])
    print (x,business_rating_per_topic[x])

(u'fY3lvNKf5KyQUDwUz2apnQ', {0: 1.0, 1: 0.3333333333333333, 2: 0.0, 3: 1.0, 4: 0.0})
(u'Qy26C1ewCj3GlQx7B-FAcw', {0: 0.5, 1: 0.0, 2: 0.0, 3: 0.3333333333333333, 4: 0.0})
(u'saQQmDQQJ14BVkm_sg53UA', {0: 1.0, 1: 0.8571428571428571, 2: 0.0, 3: 1.0, 4: 0.0})
(u'bud7jZeGsrzv_3un3ov-mw', {0: 0.0, 1: 0.0, 2: 0.0, 3: 1.0, 4: 0.0})
(u'OwZM33ZI8LZVknaLICWIhw', {0: 0.0, 1: 1.0, 2: 0.0, 3: 0.0, 4: 0.0})
(u'QH8B92lZXtLwQV9Fdrpukg', {0: 1.0, 1: 0.6666666666666666, 2: 1.0, 3: 0.0, 4: 0.0})
(u'FcJ7DkL-UKj8RLUi08lyfg', {0: 0.0, 1: 0.3333333333333333, 2: 0.0, 3: 0.0, 4: 0.0})
(u'7T_2djdY6ldRugsuWb2S-A', {0: 0.0, 1: 0.8333333333333334, 2: 0.0, 3: 1.0, 4: 0.0})
(u'dc2jT85aZGNguurhIwJDbA', {0: 0.3333333333333333, 1: 0.16666666666666666, 2: 1.0, 3: 0.5, 4: 0.0})
(u'Nz9Tk-YjE6RHQBOExYbNng', {0: 1.0, 1: 1.0, 2: 0.0, 3: 0.0, 4: 0.0})
(u'16tWXenJ1wTqugXJU7yiZg', {0: 0.5, 1: 0.16666666666666666, 2: 0.0, 3: 0.0, 4: 0.0})
(u'NTKxxbg2l55sdKD1UmBiBw', {0: 0.25, 1: 0.3333333333333333, 2: 1.0, 3: 0.0, 4: 0.0})
(u'ZDDz

In [12]:
# get average rating per user
users_rating_per_topic = defaultdict(dict)
for x in users:
    users_rating_per_topic[x] =   rating_per_business(users[x])
    print (x,users_rating_per_topic[x] )
    

(u'AJVIsIzznYPDfuKm7tBg1Q', {0: 1.0, 1: 0.6, 2: 0.0, 3: 0.0, 4: 0.0})
(u'bhpl4CYM2C1S_hEwZOqAfA', {0: 0.6666666666666666, 1: 0.7, 2: 0.0, 3: 0.0, 4: 0.0})
(u'ZuYuXMec3RkzrX4g3fE_ZA', {0: 0.6, 1: 0.14285714285714285, 2: 1.0, 3: 0.0, 4: 0.0})
(u'LIV6TiZSTWnRJKBZGnZuRQ', {0: 0.5, 1: 0.6486486486486487, 2: 0.0, 3: 0.8333333333333334, 4: 0.0})
(u'_2qsWUeMwqDliDz4RZuOTA', {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0})
(u'5fAaBl0dC69kW5untTf8KQ', {0: 0.6, 1: 0.5, 2: 0.0, 3: 0.0, 4: 0.0})
(u'MVj_1U4D5h7iDtu5TG0d2Q', {0: 1.0, 1: 0.8181818181818182, 2: 0.0, 3: 0.0, 4: 0.0})
(u'hvZ5o9YNJJ-TVCcdT42VoQ', {0: 0.7755102040816326, 1: 0.6666666666666666, 2: 0.8181818181818182, 3: 1.0, 4: 0.0})
(u'mFZFkcgtF7KaOAzLwpdKfw', {0: 0.0, 1: 0.75, 2: 0.0, 3: 0.0, 4: 0.0})
(u'9LetdklTpcKWXAy-mYWCaQ', {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 0.0})
(u'm3AqU6kLMDai5M2HBCcX4Q', {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0})
(u'OqWWcrmQm3ZGhVXi55ve6Q', {0: 0.0, 1: 1.0, 2: 0.0, 3: 1.0, 4: 0.0})
(u'TSfeZ_2RTfWQVoW6jTcbcA', {0: 0.5, 1

In [13]:
from pyspark import SparkContext
sc = SparkContext("local", "Recommend")

In [14]:
# train model per topic
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
rank = 10
numIterations = 10
topic_model = {}
# train each model using topic-specific data and store in topic_model
for topic in range(5):
    ratings = users_business_topic_rating[topic][:1200]
    ratings = sc.parallelize(ratings)
    model = ALS.train(ratings, rank, numIterations)
    topic_model[topic] = model
    
    

In [15]:
# compute MSE per topic
for topic in range(5):
    # retrieve test data for the topic
    test_ratings = users_business_topic_rating[topic][1200:]
    test_ratings = sc.parallelize(test_ratings)
    # format testdata to get (user_id, business_id)
    testdata = test_ratings.map(lambda x: (x[0], x[1]))
    # retrieve topic specific trained model
    model = topic_model[topic]
    # predict
    predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
    ratesAndPreds = test_ratings.map(lambda r: ((r[0], r[1]), r[2])).leftOuterJoin(predictions)
    # take the average rating of a user in case there is no prediction (when the user has not been seen in the training data)
    ratesAndPreds =ratesAndPreds.map(lambda r:((r[0][0], r[0][1]), (r[1][0],r[1][1] or users_rating_per_topic[id_user_map[r[0][0]]][topic]) ))
    # calculate MSE
    MSE = (ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    print("Mean Squared Error = " + str(MSE))


Mean Squared Error = 0.199996461984
Mean Squared Error = 0.098047108038
Mean Squared Error = 0.256336342723
Mean Squared Error = 0.248721319367
Mean Squared Error = 0.0


In [16]:
# predict for specific existing user

import operator
new_user = 'qI9WTIXOi2OTSG4bsc55fw'
print(new_user)
avg_topic_rating_for_user = users_rating_per_topic[new_user]
selected_topic = max(avg_topic_rating_for_user.iteritems(), key=operator.itemgetter(1))[0]
# select model based on average rating of user (aka which topic the user prefers the most)
selected_model = topic_model[selected_topic]

qI9WTIXOi2OTSG4bsc55fw


In [17]:
# rate all businesses using above model and pick top 10

test_data_for_selected_user = []

for business in id_business_map:
    test_data_for_selected_user.append((user_id_map[new_user], business))
test_data_for_selected_user = sc.parallelize(test_data_for_selected_user)
predictions = selected_model.predictAll(test_data_for_selected_user)#.map(lambda r: ((r[0], r[1]), r[2]))

In [19]:
# sort predictions
sorted_predictions = sorted(predictions.collect(),key = lambda x: x[1], reverse = True)
sorted_predictions

[Rating(user=193, product=1151, rating=0.0),
 Rating(user=193, product=1150, rating=-0.47938723642673275),
 Rating(user=193, product=1149, rating=-0.3595404398108507),
 Rating(user=193, product=1148, rating=-0.47938723642673275),
 Rating(user=193, product=1147, rating=0.0),
 Rating(user=193, product=1146, rating=0.0),
 Rating(user=193, product=1145, rating=-0.3195915009823197),
 Rating(user=193, product=1144, rating=0.0),
 Rating(user=193, product=1143, rating=-0.47938723642673275),
 Rating(user=193, product=1142, rating=0.0),
 Rating(user=193, product=1141, rating=-0.47938723642673275),
 Rating(user=193, product=1140, rating=-0.47938723642673275),
 Rating(user=193, product=1139, rating=0.0),
 Rating(user=193, product=1138, rating=0.0),
 Rating(user=193, product=1137, rating=0.0),
 Rating(user=193, product=1136, rating=-0.3344479352508911),
 Rating(user=193, product=1135, rating=0.0),
 Rating(user=193, product=1134, rating=0.0),
 Rating(user=193, product=1133, rating=0.0),
 Rating(user

In [20]:
# predict top 10
for x in sorted_predictions[:10]:
    print (id_user_map[x[0]],id_business_map[x[1]], x[2])


(u'qI9WTIXOi2OTSG4bsc55fw', u'U_x3Ryer88VwUlqa71Np-g', 0.0)
(u'qI9WTIXOi2OTSG4bsc55fw', u'aVkVDJ01OgvNwhlCUz_bng', -0.47938723642673275)
(u'qI9WTIXOi2OTSG4bsc55fw', u'HI56ArpYEbhEUeRx9Curpw', -0.3595404398108507)
(u'qI9WTIXOi2OTSG4bsc55fw', u'6YrmJ_pxLSTabcM_FFZhlQ', -0.47938723642673275)
(u'qI9WTIXOi2OTSG4bsc55fw', u'RA1Jchnz3uQDg4YVHckKXQ', 0.0)
(u'qI9WTIXOi2OTSG4bsc55fw', u'mJHoOJH10A-cjtRdQCV6ow', 0.0)
(u'qI9WTIXOi2OTSG4bsc55fw', u'tnG4_L5bZiffTbaFru9EBg', -0.3195915009823197)
(u'qI9WTIXOi2OTSG4bsc55fw', u'CvGfyePXPjKt3btUj1h9uQ', 0.0)
(u'qI9WTIXOi2OTSG4bsc55fw', u'UY13CTgUqriT-fMxT2APLw', -0.47938723642673275)
(u'qI9WTIXOi2OTSG4bsc55fw', u'4K3EHSn_DHjdxUEoYos95Q', 0.0)
